# EXERCISE 3 - ENERGINET OPTIMAL POWER FLOW PROBLEM

## Imports

In [10]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np

## Statement

We consider a 3-bus power system comprising 3 generators: G1 at node n1 , G2 at node n2 ,
G3 at node n3 , and 1 inflexible load: D1 at node n3.

   - c: cost
   - q: max capacity
   - x: reactance
   - Load: inflexible load
  - p: production

Costs 3 generators

In [11]:
G1_c = 70
G2_c = 15
G3_c = 150

Max capacity 3 generators

In [12]:
G1_q = 150
G2_q = 150
G3_q = 150

Inflexible load

In [13]:
Load = 200

Reactance 3 lines

In [14]:
L12_x = 0.4
L23_x = 0.4
L13_x = 0.4

Max flow 3 lines

In [15]:
L12_q = 150
L23_q = 150
L13_q = 150

## Resolution

### a)

Formulate the optimal power flow problem in this system, using the DC linearization
of the power flow equations. Specify the number of variables and constraints.

In [17]:
model = gp.Model("Ex3")

#Variables generator productions
G1_p = model.addVar(name='Production Generator 1')
G2_p = model.addVar(name='Production Generator 2')
G3_p = model.addVar(name='Production Generator 3')

#Variables node angles
n1_a = model.addVar(name='Angle node 1')
n2_a = model.addVar(name='Angle node 2')
n3_a = model.addVar(name='Angle node 3')


#Constrains nodes
n1_cons = model.addLConstr(1/(L12_x)*(n1_a - n2_a) + 1/(L13_x)*(n1_a - n3_a), GRB.EQUAL, G1_p)
n2_cons = model.addLConstr(1/(L12_x)*(n2_a - n1_a) + 1/(L23_x)*(n2_a - n3_a), GRB.EQUAL, G2_p)
n3_cons = model.addLConstr(1/(L13_x)*(n3_a - n1_a) + 1/(L23_x)*(n3_a - n2_a) + Load, GRB.EQUAL, G3_p)

#Constrains lines
L12_consUP = model.addLConstr(1/(L12_x)*(n1_a - n2_a), GRB.LESS_EQUAL, L12_q)
L12_consDOWN = model.addLConstr(1/(L12_x)*(n1_a - n2_a), GRB.GREATER_EQUAL, -L12_q)
L23_consUP = model.addLConstr(1/(L23_x)*(n2_a - n3_a), GRB.LESS_EQUAL, L23_q)
L23_consDOWN = model.addLConstr(1/(L23_x)*(n2_a - n3_a), GRB.GREATER_EQUAL, -L23_q)
L13_consUP = model.addLConstr(1/(L13_x)*(n1_a - n3_a), GRB.LESS_EQUAL, L13_q)
L13_consDOWN = model.addLConstr(1/(L13_x)*(n1_a - n3_a), GRB.GREATER_EQUAL, -L13_q)

#Constrains Generator max power
G1_consUP = model.addLConstr(G1_p, GRB.GREATER_EQUAL, 0)
G1_consDOWN = model.addLConstr(G1_p, GRB.LESS_EQUAL, G1_q)
G2_consUP = model.addLConstr(G2_p, GRB.GREATER_EQUAL, 0)
G2_consDOWN = model.addLConstr(G2_p, GRB.LESS_EQUAL, G2_q)
G3_consUP = model.addLConstr(G3_p, GRB.GREATER_EQUAL, 0)
G3_consDOWN = model.addLConstr(G3_p, GRB.LESS_EQUAL, G3_q)

#Constrains nodes angles
#n1_a_consUP = model.addLConstr(n1_a, GRB.GREATER_EQUAL, - np.pi)
#n1_a_consDOWN = model.addLConstr(n1_a, GRB.LESS_EQUAL, np.pi)
n2_a_consUP = model.addLConstr(n2_a, GRB.GREATER_EQUAL, - np.pi)
n2_a_consDOWN = model.addLConstr(n2_a, GRB.LESS_EQUAL, np.pi)
n3_a_consUP = model.addLConstr(n3_a, GRB.GREATER_EQUAL, - np.pi)
n3_a_consDOWN = model.addLConstr(n3_a, GRB.LESS_EQUAL, np.pi)

#Constrain reference angle, reference angle chosen n1_a
ref_a_cons = model.addLConstr(n1_a, GRB.EQUAL, 0)


#Objective Function
model.setObjective(G1_c*G1_p + G2_c*G2_p + G3_c*G3_p, GRB.MINIMIZE)

model.optimize()

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: AMD Ryzen 7 7840U with Radeon 780M Graphics, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 20 rows, 6 columns and 35 nonzeros
Model fingerprint: 0x87891033
Coefficient statistics:
  Matrix range     [1e+00, 5e+00]
  Objective range  [2e+01, 2e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+00, 2e+02]
Presolve time: 0.00s

Solved in 0 iterations and 0.00 seconds (0.00 work units)
Infeasible model


In [9]:
if model.status == GRB.OPTIMAL:
    optimal_objective = model.ObjVal
    optimal_x_G1 = x_G1.x
    optimal_x_G2 = x_G2.x
    optimal_x_G3 = x_G3.x
    optimal_dual_1 = constraint_1.Pi
    optimal_dual_2 = constraint_2.Pi
    optimal_dual_3 = constraint_3.Pi
    optimal_dual_4 = constraint_4.Pi
    print(f"optimal objective: {optimal_objective}")
    print(f"optimal value of {x_G1.VarName}: {optimal_x_G1}")
    print(f"optimal value of {x_G2.VarName}: {optimal_x_G2}")
    print(f"optimal value of {x_G3.VarName}: {optimal_x_G3}")
else:
    print(f"optimization of {model.ModelName} was not successful")

3.141592653589793

In [ ]:
pi